In [ ]:
#https://wiki.openstreetmap.org/wiki/Main_Page
import geocoder
import requests
from bs4 import BeautifulSoup
import yaml
from aoigeos import io
from aoigeos import converters as cv

In [ ]:
with open("../env.yml", 'r') as stream:
    try:
        print(yaml.safe_load(stream))
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
def geocode_osm_polygon(location_str,max_return=1000):
    g = geocoder.osm(location_str,maxRows=max_return)
    if g.ok:
        gj = g.geojson
        i = 0
        rank = {'loc':i,'importance':0}
        for f in gj['features']:
            if f['properties']['importance'] > rank['importance']:
                rank.update({'loc':i,'importance':f['properties']['importance']})
        return gj['features'][rank['loc']]
    else:
        return {'status':'no_match','location_str':location_str}

def geocode_geocodefarm_polygon(location_str):
    g = geocoder.geocodefarm(location_str)
    if g.ok:
        gj = g.geojson                
        bb = gj['features'][0]['properties']['bbox']        
        geom = bbox_to_geometry(gj['features'][0]['properties']['bbox'])
        gj['features'][0]['geometry'].update(geom)
        return gj
    else:
        return {'status':'no_match','location_str':location_str}

def geocode_bing_polygon(location_str,bing_key):
    g = geocoder.bing(location_str,key=bing_key)
    if g.ok:
        gj = g.geojson                
        return gj
    else:
        return {'status':'no_match','location_str':location_str}

    
class Geocode():
    
    def osm_all_points(self,location_str,max_return=1000):
        g = geocoder.osm(location_str,maxRows=max_return)
        if g.ok:
            self.geojson_point = g.geojson
        else:
            self.geojson_point = {'status':'no_match','location_str':location_str}

    def osm_best_point(self,location_str,max_return=1000):
        self.osm_all_points(location_str,max_return=1000)
        i = 0
        rank = {'loc':i,'importance':0}
        for f in self.geojson_point['features']:
            if f['properties']['importance'] > rank['importance']:
                rank.update({'loc':i,'importance':f['properties']['importance']})
        winner = self.geojson_point['features'][rank['loc']]
        self.geojson_point = cv.geojson_from_features([winner])


In [ ]:
gc = Geocode()
gc.osm_all_points('new york city')

In [ ]:
gc = Geocode()
gc.osm_best_point('new york city')

In [ ]:
page = requests.get('https://data.mongabay.com/cities_pop_01.htm')
content = BeautifulSoup(page.text,'html.parser')
city_table = content.find(class_='boldtable')
city_recs = city_table.find_all('tr')
i = 0
geojson = {"type": "FeatureCollection","features": []}
errors = []
for c in city_recs:
    if i > 1:
        city_str = c.contents[0].text.lower()
        gj = geocode_bing_polygon(city_str)
        if gj['status'] != 'no_match':
            geojson['features'].append(gj['features'][0])
        else:
            errors.append(city_str)
    i += 1
    if i % 100 == 0:
        print(i)

In [ ]:
city_recs = read_json('/Users/michaeln/Documents/data/archipelago/aois/cities.json')

In [ ]:
quality_list = ['AdminDivision1','PopulatedPlace']
i = 0
geojson = {"type": "FeatureCollection","features": []}
errors = []
for c in city_recs['features']:
    city_str = c['properties']['name']
    city_gj = geocode_bing_polygon(city_str)
    if 'status' not in city_gj.keys():
        if gj['features'][0]['properties']['quality'] in quality_list:
            geojson['features'].append(gj['features'][0])
        else:
            errors.append(city_str)
    else:
        errors.append(city_str)
    i += 1
    if i % 100 == 0:
        print(i)


In [ ]:
write_json(geojson,'/Users/michaeln/Documents/data/archipelago/aois/cities-3-bing.json')

In [ ]:
len(errors)

In [ ]:
errors

In [ ]:
i=0
for f in geojson['features']:
    if f['properties']['quality']!='PopulatedPlace':
        i+=1
print(i)

In [ ]:
f

In [ ]:
a = geocode_bing_polygon('seoul, south korea')

In [ ]:
type(a)

In [ ]:
quality_list = ['AdminDivision1','PopulatedPlace']

In [ ]:
'PopulatedPlace' in quality_list